In [177]:
import sqlite3
import pandas as pd
import io
import re
import ssl
from urllib.request import Request,urlopen
import requests
import json
import numpy as np
from bs4 import BeautifulSoup

In [178]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [179]:
url2='https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names'
req=Request(url2,headers={'User-Agent':'Mozila/5.0'})
retdt=urlopen(req).read().decode('utf-8')



In [180]:
lst=retdt.replace('\n','').replace('.',' ').split(" ")
lstcolnames=list()
for dt  in lst:   
    if(re.search('^[a-z].+:',dt)):
         if(re.search('^[a-z].+:[|]',dt) or re.search('^[a-z].+://',dt) or re.search('^[a-z]-[a-z].+:',dt)):
                continue
         else:
                lstcolnames.append(dt.replace(':',''))
for dtt in lst:
    if(re.search('^>[0-9].+',dtt)):
        lstcolnames.append('class')
print(lstcolnames)
  

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class']


In [181]:
df=pd.DataFrame()

In [182]:
url='https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
rq=requests.get(url).content
df=pd.read_csv(io.StringIO(rq.decode('utf-8')))

In [183]:
df.columns=lstcolnames

In [184]:
df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [185]:
con=sqlite3.connect('sqladb.sqlite')

In [186]:
df.iloc[1,1]

' Private'

In [187]:
cur=con.cursor()

In [188]:
cur.execute('DROP TABLE IF EXISTS AdultData')

In [189]:
cur.execute('CREATE TABLE AdultData (age INT, workclass TEXT, fnlwgt INT, education TEXT, education_num INT , marital_status TEXT, occupation TEXT, relationship TEXT, race TEXT, sex TEXT, capital_gain INT, capital_loss INT, hours_per_week INT, native_country TEXT, class TEXT)')

In [190]:
for i in range(len(df.index)):
    cur.execute('''INSERT INTO AdultData(age, workclass, fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,
    hours_per_week,native_country,class) 
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
    (int(df.loc[i]['age']),df.loc[i]['workclass'],int(df.loc[i]['fnlwgt']),df.loc[i]['education'],
     int(df.loc[i]['education-num']),df.loc[i]['marital-status'],df.loc[i]['occupation'],df.loc[i]['relationship'],
     df.loc[i]['race'],df.loc[i]['sex'],int(df.loc[i]['capital-gain']),int(df.loc[i]['capital-loss']),
     int(df.loc[i]['hours-per-week']),df.loc[i]['native-country'],df.loc[i]['class']))
con.commit()

In [191]:
sqlstr='SELECT * FROM AdultData LIMIT 10'

In [192]:
cur.execute(sqlstr)
rows=cur.fetchall()
for row in rows:
    print(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14])

50  Self-emp-not-inc 83311  Bachelors 13  Married-civ-spouse  Exec-managerial  Husband  White  Male 0 0 13  United-States  <=50K
38  Private 215646  HS-grad 9  Divorced  Handlers-cleaners  Not-in-family  White  Male 0 0 40  United-States  <=50K
53  Private 234721  11th 7  Married-civ-spouse  Handlers-cleaners  Husband  Black  Male 0 0 40  United-States  <=50K
28  Private 338409  Bachelors 13  Married-civ-spouse  Prof-specialty  Wife  Black  Female 0 0 40  Cuba  <=50K
37  Private 284582  Masters 14  Married-civ-spouse  Exec-managerial  Wife  White  Female 0 0 40  United-States  <=50K
49  Private 160187  9th 5  Married-spouse-absent  Other-service  Not-in-family  Black  Female 0 0 16  Jamaica  <=50K
52  Self-emp-not-inc 209642  HS-grad 9  Married-civ-spouse  Exec-managerial  Husband  White  Male 0 0 45  United-States  >50K
31  Private 45781  Masters 14  Never-married  Prof-specialty  Not-in-family  White  Female 14084 0 50  United-States  >50K
42  Private 159449  Bachelors 13  Married-ci

In [193]:
sqlstr1="SELECT avg(hours_per_week) FROM AdultData WHERE workclass like '%Private' "
cur.execute(sqlstr1)
avg=cur.fetchone()
avg[0]

40.267095523440254

In [194]:
Frequency=pd.Series()
Education=pd.Series()
dfedu=pd.DataFrame()
#dfedu.columns=['Frequence','Education']
sqlstr2="SELECT education,COUNT(education) AS Frequency  FROM AdultData GROUP BY education "
cur.execute(sqlstr2)
fred=cur.fetchall()
lstdt=list()
lstedu=list()
for vl in fred:
    print(vl[0],vl[1])
#np
#dfedu.transpose()
#dfedu.head()
#dfedu.append(pd.Series(lstedu),ignore_index=True)
    
#dfedu.columns=['Fequency','Education']

 10th 933
 11th 1175
 12th 433
 1st-4th 168
 5th-6th 333
 7th-8th 646
 9th 514
 Assoc-acdm 1067
 Assoc-voc 1382
 Bachelors 5354
 Doctorate 413
 HS-grad 10501
 Masters 1723
 Preschool 51
 Prof-school 576
 Some-college 7291


In [195]:
sqlstr3='SELECT occupation,COUNT(occupation) as FreqOccup FROM AdultData GROUP BY occupation '
cur.execute(sqlstr3)
freoc=cur.fetchall()
for fq in freoc:
    if str(fq[0])==' ?':
        Occup='Unknown'
    else:
        Occup=fq[0]
    print(Occup,fq[1])

Unknown 1843
 Adm-clerical 3769
 Armed-Forces 9
 Craft-repair 4099
 Exec-managerial 4066
 Farming-fishing 994
 Handlers-cleaners 1370
 Machine-op-inspct 2002
 Other-service 3295
 Priv-house-serv 149
 Prof-specialty 4140
 Protective-serv 649
 Sales 3650
 Tech-support 928
 Transport-moving 1597


In [196]:
sqlstr4='SELECT relationship,COUNT(relationship) FROM AdultData GROUP BY relationship'
cur.execute(sqlstr3)
fqrel=cur.fetchall()
for fq2 in fqrel:
    if fq2[0]==' ?':
        rel='Unknown'
    else:
        rel=fq2[0]
    print(rel,fq[1])

Unknown 1597
 Adm-clerical 1597
 Armed-Forces 1597
 Craft-repair 1597
 Exec-managerial 1597
 Farming-fishing 1597
 Handlers-cleaners 1597
 Machine-op-inspct 1597
 Other-service 1597
 Priv-house-serv 1597
 Prof-specialty 1597
 Protective-serv 1597
 Sales 1597
 Tech-support 1597
 Transport-moving 1597


In [197]:
sqlstr4="SELECT * FROM AdultData WHERE marital_status like ' Married%' and workclass like ' Private%' and  education like ' Masters%'"
cur.execute(sqlstr4)
reqdt=cur.fetchall()
cnt=0
for dt in reqdt:
    cnt=cnt+1
    print(dt)
print('There are {} people who are married, working in private sector and having a masters degree'.format(cnt))

    

(37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K')
(33, ' Private', 202051, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' White', ' Male', 0, 0, 50, ' United-States', ' <=50K')
(76, ' Private', 124191, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 40, ' United-States', ' >50K')
(31, ' Private', 99928, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' White', ' Female', 0, 0, 50, ' United-States', ' <=50K')
(35, ' Private', 138992, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Other-relative', ' White', ' Male', 7298, 0, 40, ' United-States', ' >50K')
(34, ' Private', 142897, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' Asian-Pac-Islander', ' Male', 7298, 0, 35, ' Taiwan', ' >50K')
(62, ' Private', 270092, ' Masters', 14, ' Married-civ-spouse', ' Prof-s

In [198]:
sqlstr5="SELECT workclass,avg(age) As Average_Age,min(age) As Min_Age,max(age) As Max_Age FROM  AdultData WHERE workclass NOT LIKE ' Without-pay%' GROUP BY workclass"
cur.execute(sqlstr5)
reqdt2=cur.fetchall()
for dt3 in reqdt2:
    if dt3[0]!=' ?':
        print(dt3[0],dt3[1],dt3[2],dt3[3])

 Federal-gov 42.590625 17 90
 Local-gov 41.75107501194458 17 90
 Never-worked 20.571428571428573 17 30
 Private 36.797585477617204 17 90
 Self-emp-inc 46.01702508960574 17 84
 Self-emp-not-inc 44.96969696969697 17 90
 State-gov 39.43639167309175 17 81


In [199]:
dft=pd.DataFrame()
sqlstr6="SELECT age,native_country FROM AdultData WHERE native_country!=' ?'"
#cur.execute(sqlstr6)
#dft=cur.fetchall()
dft=pd.read_sql_query(sqlstr6,con)
dft.head()


,age,native_country
0,50,United-States
1,38,United-States
2,53,United-States
3,28,Cuba
4,37,United-States


In [200]:
names=['Country','0-15','16-40','41-60','60+']
dfr=pd.DataFrame(columns=names)
countries=dft['native_country'].unique()
dfr['Country']=countries
dfr=dfr.fillna(0)
for ind in range(dft['native_country'].count()):
    #for inx in range(dfr['Country'].count()):
    cnt=dft.iloc[ind]['native_country']
    inx=0
    nsum=0
    if  0<int(dft.iloc[ind]['age'])<=15:
        inx=dfr[dfr['Country']==cnt].index#=rows['native_country']]['0-15']+1
    elif 15<int(dft['age'].iloc[ind])<=40:
        inx=dfr[dfr['Country']==cnt].index
        nsum=int(dfr['16-40'].iloc[inx])+1
        dfr['16-40'].iloc[inx]=nsum
        nsum=int(dfr['60+'].iloc[inx])+1
        dfr['60+'].iloc[inx]=nsum
dfr

C:\Users\abhinav\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Country,0-15,16-40,41-60,60+
0,United-States,0,17039,9990,0
1,Cuba,0,38,44,0
2,Jamaica,0,56,23,0
3,India,0,57,41,0
4,Mexico,0,486,147,0
5,South,0,46,28,0
6,Puerto-Rico,0,58,49,0
7,Honduras,0,9,4,0
8,England,0,47,35,0
9,Canada,0,60,43,0


Task 2:

Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data


In [204]:
df2=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')

In [205]:
import sqlalchemy
from sqlalchemy import create_engine

In [206]:
engine=create_engine('sqlite:///:memory:',echo=True)

In [207]:
from sqlalchemy.ext.declarative import declarative_base

In [208]:
Base=declarative_base()

In [209]:
lstcolnames=['age',
 'work_class',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'classs']

In [210]:
df2.columns=lstcolnames

In [211]:
df2.head()

,age,work_class,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,classs
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


1. Create an sqlalchemy engine using a sample from the data set

In [212]:
from sqlalchemy import Column,Integer,String

In [213]:
class Adultdt(Base):
    __tablename__='AdultData'
    id=Column(Integer,primary_key=True)
    age=Column(Integer)
    work_class=Column(String)
    fnlwgt=Column(Integer)
    education=Column(String)
    education_num=Column(Integer)
    marital_status=Column(String)
    occupation=Column(String)
    relationship=Column(String)
    race=Column(String)
    sex=Column(String)
    capital_gain=Column(String)
    capital_loss=Column(Integer)
    hours_per_week=Column(String)
    native_country=Column(String)
    classs=Column(String)
    
    def __repr__(self):
         return "<AdultData(id='%s', age='%s', work_class='%s', fnlwgt='%s', education='%s', education_num='%s',marital_status='%s', occupation='%s', relationship='%s', race='%s',sex='%s',capital_gain='%s', capital_loss='%s',hours_per_week='%s', native_country='%s', classs='%s')>" % (
                            self.id,self.age, self.work_class, self.fnlwgt, self.education, self.education_num, self.marital_status, self.occupation, self.relationship, self.race, self.sex,self.capital_gain, self.capital_loss,self.hours_per_week,self.native_country,self.classs)
         
        
    

In [214]:
Adultdt.__table__

Table('AdultData', MetaData(bind=None), Column('id', Integer(), table=<AdultData>, primary_key=True, nullable=False), Column('age', Integer(), table=<AdultData>), Column('work_class', String(), table=<AdultData>), Column('fnlwgt', Integer(), table=<AdultData>), Column('education', String(), table=<AdultData>), Column('education_num', Integer(), table=<AdultData>), Column('marital_status', String(), table=<AdultData>), Column('occupation', String(), table=<AdultData>), Column('relationship', String(), table=<AdultData>), Column('race', String(), table=<AdultData>), Column('sex', String(), table=<AdultData>), Column('capital_gain', String(), table=<AdultData>), Column('capital_loss', Integer(), table=<AdultData>), Column('hours_per_week', String(), table=<AdultData>), Column('native_country', String(), table=<AdultData>), Column('classs', String(), table=<AdultData>), schema=None)

In [215]:
Base.metadata.create_all(engine)

2019-01-16 22:48:44,491 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-16 22:48:44,494 INFO sqlalchemy.engine.base.Engine ()
2019-01-16 22:48:44,496 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-16 22:48:44,499 INFO sqlalchemy.engine.base.Engine ()
2019-01-16 22:48:44,503 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("AdultData")
2019-01-16 22:48:44,506 INFO sqlalchemy.engine.base.Engine ()
2019-01-16 22:48:44,510 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "AdultData" (
	id INTEGER NOT NULL, 
	age INTEGER, 
	work_class VARCHAR, 
	fnlwgt INTEGER, 
	education VARCHAR, 
	education_num INTEGER, 
	marital_status VARCHAR, 
	occupation VARCHAR, 
	relationship VARCHAR, 
	race VARCHAR, 
	sex VARCHAR, 
	capital_gain VARCHAR, 
	capital_loss INTEGER, 
	hours_per_week VARCHAR, 
	native_country VARCHAR, 
	classs VARCHAR, 
	PRIMARY KEY (id)
)


2019-01-16 22:48:44,511 INFO sql

In [216]:
df2.to_sql(name='AdultData',con=engine,if_exists='append',index=False)

2019-01-16 22:48:45,432 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("AdultData")
2019-01-16 22:48:45,432 INFO sqlalchemy.engine.base.Engine ()
2019-01-16 22:48:45,467 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-16 22:48:46,237 INFO sqlalchemy.engine.base.Engine INSERT INTO "AdultData" (age, work_class, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, native_country, classs) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-01-16 22:48:46,238 INFO sqlalchemy.engine.base.Engine ((50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K'), (38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K'), (53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husb

In [217]:
from sqlalchemy.orm import sessionmaker

In [218]:
Session=sessionmaker(bind=engine)

In [219]:
session=Session()

2. Write two basic update queries

In [220]:
updat1=session.query(Adultdt).filter_by(id='50').first()

2019-01-16 22:48:48,489 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-16 22:48:48,494 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ?
 LIMIT ? OFFSET ?
2019-01-16 22:48:48,497 INFO 

In [221]:
query2 = session.query(Adultdt).filter(Adultdt.id=='50').order_by(Adultdt.id)

In [222]:
query2.all()

2019-01-16 22:48:49,594 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ? ORDER BY "AdultData".id
2019-01-16 22:48:49,597 INFO sqlalchemy.engine.base.Engine ('50',)


[<AdultData(id='50', age='25', work_class=' Private', fnlwgt='32275', education=' Some-college', education_num='10',marital_status=' Married-civ-spouse', occupation=' Exec-managerial', relationship=' Wife', race=' Other',sex=' Female',capital_gain='0', capital_loss='0',hours_per_week='40', native_country=' United-States', classs=' <=50K')>]

In [223]:
print(updat1.age)

25


In [224]:
updat1.native_country='Jamaica'

In [225]:
session.commit()

2019-01-16 22:48:50,440 INFO sqlalchemy.engine.base.Engine UPDATE "AdultData" SET native_country=? WHERE "AdultData".id = ?
2019-01-16 22:48:50,443 INFO sqlalchemy.engine.base.Engine ('Jamaica', 50)
2019-01-16 22:48:50,446 INFO sqlalchemy.engine.base.Engine COMMIT


In [226]:
query2.all()

2019-01-16 22:48:51,837 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-16 22:48:51,840 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ? ORDER BY "AdultData".id
2019-01-16 22:48:51,841

[<AdultData(id='50', age='25', work_class=' Private', fnlwgt='32275', education=' Some-college', education_num='10',marital_status=' Married-civ-spouse', occupation=' Exec-managerial', relationship=' Wife', race=' Other',sex=' Female',capital_gain='0', capital_loss='0',hours_per_week='40', native_country='Jamaica', classs=' <=50K')>]

In [227]:
query3 = session.query(Adultdt).filter(Adultdt.id=='60').order_by(Adultdt.id)

In [228]:
query3.all()

2019-01-16 22:48:52,379 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ? ORDER BY "AdultData".id
2019-01-16 22:48:52,381 INFO sqlalchemy.engine.base.Engine ('60',)


[<AdultData(id='60', age='30', work_class=' Private', fnlwgt='59496', education=' Bachelors', education_num='13',marital_status=' Married-civ-spouse', occupation=' Sales', relationship=' Husband', race=' White',sex=' Male',capital_gain='2407', capital_loss='0',hours_per_week='40', native_country=' United-States', classs=' <=50K')>]

In [229]:
updat2=session.query(Adultdt).filter_by(id='60').first()

2019-01-16 22:48:53,051 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ?
 LIMIT ? OFFSET ?
2019-01-16 22:48:53,053 INFO sqlalchemy.engine.base.Engine ('60', 1, 0)


In [230]:
updat2.marital_status='divorced'

In [231]:
session.commit()

2019-01-16 22:48:53,578 INFO sqlalchemy.engine.base.Engine UPDATE "AdultData" SET marital_status=? WHERE "AdultData".id = ?
2019-01-16 22:48:53,579 INFO sqlalchemy.engine.base.Engine ('divorced', 60)
2019-01-16 22:48:53,581 INFO sqlalchemy.engine.base.Engine COMMIT


In [232]:
query3.all()

2019-01-16 22:48:54,500 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-16 22:48:54,503 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ? ORDER BY "AdultData".id
2019-01-16 22:48:54,506

[<AdultData(id='60', age='30', work_class=' Private', fnlwgt='59496', education=' Bachelors', education_num='13',marital_status='divorced', occupation=' Sales', relationship=' Husband', race=' White',sex=' Male',capital_gain='2407', capital_loss='0',hours_per_week='40', native_country=' United-States', classs=' <=50K')>]

3. Write two delete queries

In [288]:
query4 = session.query(Adultdt).filter(Adultdt.id=='12').order_by(Adultdt.id)

In [289]:
query4.first()

2019-01-16 22:53:24,225 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ? ORDER BY "AdultData".id
 LIMIT ? OFFSET ?
2019-01-16 22:53:24,227 INFO sqlalchemy.engine.base.Engine ('12', 1, 0)


<AdultData(id='12', age='23', work_class=' Private', fnlwgt='122272', education=' Bachelors', education_num='13',marital_status=' Never-married', occupation=' Adm-clerical', relationship=' Own-child', race=' White',sex=' Female',capital_gain='0', capital_loss='0',hours_per_week='30', native_country=' United-States', classs=' <=50K')>

In [292]:
dell=session.query(Adultdt).filter_by(id='12').delete()
session.commit()

2019-01-16 22:54:00,255 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ?
2019-01-16 22:54:00,257 INFO sqlalchemy.engine.base.Engine (50,)
2019-01-16 22:54:00,261 INFO sqlalchemy.engine.base.Engine

In [294]:
query4.all()

2019-01-16 22:54:11,304 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".id = ? ORDER BY "AdultData".id
2019-01-16 22:54:11,307 INFO sqlalchemy.engine.base.Engine ('12',)


[]

In [238]:
query5=session.query(Adultdt).filter(id=='17')

In [239]:
query5.first()

2019-01-16 22:48:57,907 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE 0 = 1
 LIMIT ? OFFSET ?
2019-01-16 22:48:57,908 INFO sqlalchemy.engine.base.Engine (1, 0)


In [240]:
del2=session.query(Adultdt).filter(id=='17').delete()


2019-01-16 22:48:58,120 INFO sqlalchemy.engine.base.Engine DELETE FROM "AdultData" WHERE 0
2019-01-16 22:48:58,122 INFO sqlalchemy.engine.base.Engine ()


In [241]:
session.commit()

2019-01-16 22:48:59,693 INFO sqlalchemy.engine.base.Engine COMMIT


In [242]:
query5.all()

2019-01-16 22:48:59,914 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-16 22:48:59,917 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE 0 = 1
2019-01-16 22:48:59,920 INFO sqlalchemy.engine.base.Engine (

[]

4. Write two filter queries

In [243]:
query = session.query(Adultdt).filter(Adultdt.relationship==' Husband').order_by(Adultdt.id)

In [244]:
query.first()

2019-01-16 22:49:06,275 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".relationship = ? ORDER BY "AdultData".id
 LIMIT ? OFFSET ?
2019-01-16 22:49:06,277 INFO sqlalchemy.engine.base.Engine (' Husband',

<AdultData(id='1', age='50', work_class=' Self-emp-not-inc', fnlwgt='83311', education=' Bachelors', education_num='13',marital_status=' Married-civ-spouse', occupation=' Exec-managerial', relationship=' Husband', race=' White',sex=' Male',capital_gain='0', capital_loss='0',hours_per_week='13', native_country=' United-States', classs=' <=50K')>

In [245]:
session.query(Adultdt).filter(Adultdt.fnlwgt.in_(['83311', '215646'])).all()

2019-01-16 22:49:07,579 INFO sqlalchemy.engine.base.Engine SELECT "AdultData".id AS "AdultData_id", "AdultData".age AS "AdultData_age", "AdultData".work_class AS "AdultData_work_class", "AdultData".fnlwgt AS "AdultData_fnlwgt", "AdultData".education AS "AdultData_education", "AdultData".education_num AS "AdultData_education_num", "AdultData".marital_status AS "AdultData_marital_status", "AdultData".occupation AS "AdultData_occupation", "AdultData".relationship AS "AdultData_relationship", "AdultData".race AS "AdultData_race", "AdultData".sex AS "AdultData_sex", "AdultData".capital_gain AS "AdultData_capital_gain", "AdultData".capital_loss AS "AdultData_capital_loss", "AdultData".hours_per_week AS "AdultData_hours_per_week", "AdultData".native_country AS "AdultData_native_country", "AdultData".classs AS "AdultData_classs" 
FROM "AdultData" 
WHERE "AdultData".fnlwgt IN (?, ?)
2019-01-16 22:49:07,581 INFO sqlalchemy.engine.base.Engine ('83311', '215646')


[<AdultData(id='1', age='50', work_class=' Self-emp-not-inc', fnlwgt='83311', education=' Bachelors', education_num='13',marital_status=' Married-civ-spouse', occupation=' Exec-managerial', relationship=' Husband', race=' White',sex=' Male',capital_gain='0', capital_loss='0',hours_per_week='13', native_country=' United-States', classs=' <=50K')>,
 <AdultData(id='2', age='38', work_class=' Private', fnlwgt='215646', education=' HS-grad', education_num='9',marital_status=' Divorced', occupation=' Handlers-cleaners', relationship=' Not-in-family', race=' White',sex=' Male',capital_gain='0', capital_loss='0',hours_per_week='40', native_country=' United-States', classs=' <=50K')>,
 <AdultData(id='16474', age='54', work_class=' Self-emp-not-inc', fnlwgt='83311', education=' Prof-school', education_num='15',marital_status=' Married-civ-spouse', occupation=' Prof-specialty', relationship=' Husband', race=' White',sex=' Male',capital_gain='0', capital_loss='0',hours_per_week='30', native_countr

5. Write two function queries

In [246]:
from sqlalchemy import func

In [247]:
session.query(func.count('*')).select_from(Adultdt).scalar()

2019-01-16 22:49:09,704 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM "AdultData"
2019-01-16 22:49:09,705 INFO sqlalchemy.engine.base.Engine ('*',)


32560

In [248]:
session.query(func.count(Adultdt.id), Adultdt.education).group_by(Adultdt.education).all()

2019-01-16 22:49:10,484 INFO sqlalchemy.engine.base.Engine SELECT count("AdultData".id) AS count_1, "AdultData".education AS "AdultData_education" 
FROM "AdultData" GROUP BY "AdultData".education
2019-01-16 22:49:10,487 INFO sqlalchemy.engine.base.Engine ()


[(933, ' 10th'),
 (1175, ' 11th'),
 (433, ' 12th'),
 (168, ' 1st-4th'),
 (333, ' 5th-6th'),
 (646, ' 7th-8th'),
 (514, ' 9th'),
 (1067, ' Assoc-acdm'),
 (1382, ' Assoc-voc'),
 (5354, ' Bachelors'),
 (413, ' Doctorate'),
 (10501, ' HS-grad'),
 (1723, ' Masters'),
 (51, ' Preschool'),
 (576, ' Prof-school'),
 (7291, ' Some-college')]